In [19]:
import os, sys
import argparse
import time
import numpy as np

import xml.etree.ElementTree as ET

from sumolib import checkBinary

import traci
import traci.constants as tc

import matplotlib.pyplot as plt

In [20]:
import gym

In [21]:
if sys.platform == "win32":
    # windows, win32
    from sumolib import checkBinary
else:
    # mac, darwin
    if 'SUMO_HOME' in os.environ:
        tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
        sys.path.append(tools)
    else:
        sys.exit("please declare environment variable 'SUMO_HOME'")
    
import traci

In [22]:
import curbside
import utils

In [23]:
net_xml = "seattle.net.xml"
add_xml = "seattle.add.xml"
rou_xml = "seattle.trips.xml"

In [24]:
# class A2CAgent:
#     def __init__(self):
#         self.config = None

In [25]:
# class Policy:
#     def __init__(self, curb_ids):
#         self.agents = {}
#         for curb_id in curb_ids:
#             self.agents[curb_id] = 

In [26]:
# class Actor:
    
    
# class Critic:
    
    

In [126]:
class SeattleEnv(gym.Env):
    
    def __init__(self, gui):
        self.net_xml = "seattle.net.xml"
        self.add_xml = "seattle.add.xml"
        self.rou_xml = "seattle.trips.xml"
        self.curbs, self.curb_ids = self._init_curbs()
        self.gui = gui
        self.sim = self._init_sim(self.gui)
        
        self.step = 0
        self.control_window = 60
        
        self.reroute_total = 0
        self.reroute_dict = dict(zip(self.curb_ids, np.zeros(len(self.curb_ids), dtype=int)))
        self.reroute_vtype_dict = {curb_id:{'dlv':0, 'psg':0} for curb_id in self.curb_ids}
        
    
    def _init_curbs(self):
        road_network = utils.create_graph('seattle.net.xml')
        # create agent curbs
        curb_ids = []
        root = ET.parse(add_xml).getroot()
        for child in root.iter('additional'):
            for kid in child.iter('parkingArea'):
                curb_ids.append(kid.get('id'))
        curbs = {}
        for curb_id in curb_ids:
            curbs[curb_id] = curbside.SmartCurbside(1, 'seattle.add.xml', 'seattle.net.xml', curb_id, ['passenger', 'delivery'], road_network)
        for curb in curbs.values():
            curb.find_neighborhood(road_network, curbs)
        return curbs, curb_ids
    
    def _init_sim(self, gui):
        if sys.platform == "win32":
            if gui:
                sumoBinary = checkBinary('sumo-gui')
            else:
                sumoBinary = checkBinary('sumo')
        else:　# mac, darwin
            if gui:
                sumoBinary = "sumo-gui"
            else:
                sumoBinary = "sumo"
        traci.start([sumoBinary, "-c", "seattle.sumocfg", "--time-to-teleport", "-1"])
        return traci.getConnection()
    
    def batch(self):
        for i in range(self.control_window):
            self.sim.simulationStep()
            self.step += 1
            for curb in self.curbs.values():
                curb._update_neigh_occupy(self.curbs)
                # check if there is any vehicle enters
                v_enter = set(self.sim.edge.getLastStepVehicleIDs(curb.edge)) - curb.moving_vehicle
                for veh in v_enter:
                    if self.sim.vehicle.getNextStops(veh) \
                        and curb.id in [item[2] for item in self.sim.vehicle.getNextStops(veh)] \
                        and not self.sim.vehicle.isStopped(veh):
                        # if the trips is not ending and the current edge is the parking stop, v_leave should be excluded
                        # vclass: passenger or delivery
                        vclass = self.sim.vehicle.getVehicleClass(veh)
                        if vclass == 'delivery':
                            occ = curb.dlv_occ
                            cap = curb.dlv_cap
                        else:
                            occ = curb.psg_occ
                            cap = curb.psg_cap
                        if occ < cap:
                            # if can park, add to occupied set : planned + parked
                            curb.occupied_vehicle.add(veh)
                        else:
                            # cannot park at this edge, reroute
                            # item._reroute_choice() returns (curb_id, distance) tuple
                            self.sim.vehicle.rerouteParkingArea(veh, curb._reroute_choice(veh, self.curbs)[0])
                            
                            self.reroute_total += 1
                            self.reroute_dict[curb.id] += 1
                            if vclass == 'delivery':
                                self.reroute_vtype_dict[curb.id]['dlv'] += 1
                            else:
                                self.reroute_vtype_dict[curb.id]['psg'] += 1
                            
                # v_leave : parked vehicle of last time step - parked vehicle at this time step
                v_leave = curb.parked_vehicle - set(self.sim.parkingarea.getVehicleIDs(curb.id))
                # remove v_leave from occupied set : planned + parked
                curb.occupied_vehicle -= v_leave
                # update parked vehicle set
                curb.parked_vehicle = set(self.sim.parkingarea.getVehicleIDs(curb.id))
                # update moving vehicles on the hosting edge
                curb.moving_vehicle = set(self.sim.edge.getLastStepVehicleIDs(curb.edge))
                # update two-type occupancy for next time step
                curb._occupy_cnt()

    def control(self, actions):
        for i in range(len(self.curb_ids)):
            curb = self.curbs[self.curb_ids[i]]
            action = actions[i]
            if action == 1 and curb.psg_cap > 0 and curb.psg_occ < curb.psg_cap:
                # delivery vehicle space +1
                curb.psg_cap -= 1
                curb.dlv_cap += 1
            elif action == -1 and curb.dlv_cap > 0 and curb.dlv_occ < curb.dlv_cap:
                # passenger vehicle space +1
                curb.psg_cap += 1
                curb.dlv_cap -= 1
    
    def terminate(self):
        traci.close()
#         self.sim.close()
#         traci.close()
#     def _reroute(self):

In [127]:
class Policy:
    def __init__(self, curb_ids):
        self.curb_ids = curb_ids
        
    def forward(self, reroute_vtype_dict):
        actions = []
        for curb_id in self.curb_ids:
            reroute_dict = reroute_vtype_dict[curb_id]
            if reroute_dict['dlv'] > reroute_dict['psg']:
                action = 1 # delivery vehicle space +1
            elif reroute_dict['dlv'] < reroute_dict['psg']:
                action = -1 # passenger vehicle space +1
            else:
                action = 0
            actions.append(action)
        return actions

In [140]:
env = SeattleEnv(gui=True)
policy = Policy(env.curb_ids)
for i in range(60):
    env.batch()
    print('batch:', i, 'reroute:', env.reroute_total)
env.terminate()

batch:  0
0
batch:  1
3
batch:  2
4
batch:  3
13
batch:  4
22
batch:  5
32
batch:  6
40
batch:  7
50
batch:  8
63
batch:  9
77
batch:  10
94
batch:  11
100
batch:  12
114
batch:  13
137
batch:  14
147
batch:  15
169
batch:  16
182
batch:  17
193
batch:  18
212
batch:  19
226
batch:  20
235
batch:  21
256
batch:  22
267
batch:  23
280
batch:  24
296
batch:  25
314
batch:  26
329
batch:  27
342
batch:  28
359
batch:  29
368
batch:  30
379
batch:  31
385
batch:  32
389
batch:  33
395
batch:  34
397
batch:  35
406
batch:  36
421
batch:  37
433
batch:  38
439
batch:  39
450
batch:  40
466
batch:  41
478
batch:  42
488
batch:  43
502
batch:  44
521
batch:  45
538
batch:  46
557
batch:  47
566
batch:  48
578
batch:  49
585
batch:  50
587
batch:  51
591
batch:  52
595
batch:  53
597
batch:  54
602
batch:  55
605
batch:  56
609
batch:  57
611
batch:  58
613
batch:  59
614


In [135]:
env = SeattleEnv(gui=False)
policy = Policy(env.curb_ids)
for i in range(60):
    env.batch()
    actions = policy.forward(env.reroute_vtype_dict)
    env.control(actions)
    print('batch:', i, 'env.reroute_total:', env.reroute_total)
#     print(env.reroute_dict)
#     print(env.reroute_vtype_dict)
env.terminate()

batch:  0
0
batch:  1
2
batch:  2
3
batch:  3
5
batch:  4
9
batch:  5
15
batch:  6
16
batch:  7
21
batch:  8
22
batch:  9
22
batch:  10
24
batch:  11
25
batch:  12
28
batch:  13
31
batch:  14
31
batch:  15
31
batch:  16
32
batch:  17
36
batch:  18
37
batch:  19
42
batch:  20
43
batch:  21
43
batch:  22
46
batch:  23
47
batch:  24
50
batch:  25
53
batch:  26
53
batch:  27
54
batch:  28
59
batch:  29
60
batch:  30
62
batch:  31
65
batch:  32
66
batch:  33
70
batch:  34
72
batch:  35
75
batch:  36
78
batch:  37
78
batch:  38
81
batch:  39
84
batch:  40
85
batch:  41
85
batch:  42
88
batch:  43
89
batch:  44
92
batch:  45
95
batch:  46
97
batch:  47
99
batch:  48
100
batch:  49
101
batch:  50
102
batch:  51
103
batch:  52
105
batch:  53
105
batch:  54
106
batch:  55
108
batch:  56
108
batch:  57
111
batch:  58
111
batch:  59
112


In [139]:
traci.close()

In [123]:
def simulate(seconds = 10, gui = False):
    """
    Main function that controls the simulation
    """

    road_network = utils.create_graph('seattle.net.xml')
    
    # create agent curbs
    curb_ids = []
    root = ET.parse(add_xml).getroot()
    for child in root.iter('additional'):
        for kid in child.iter('parkingArea'):
            curb_ids.append(kid.get('id'))
    
    curbs = {}
    for curb_id in curb_ids:
        curbs[curb_id] = curbside.SmartCurbside(1, 'seattle.add.xml', 'seattle.net.xml', curb_id, ['passenger', 'delivery'], road_network)
        
    for curb in curbs.values():
        curb.find_neighborhood(road_network, curbs)
    

    reroute_num = 0
    
    if sys.platform == "win32":
        if gui:
            sumoBinary = checkBinary('sumo-gui')
        else:
            sumoBinary = checkBinary('sumo')
    
    else:
        # mac, darwin
        if gui:
            sumoBinary = "sumo-gui"
        else:
            sumoBinary = "sumo"
    
    traci.start([sumoBinary, "-c", "seattle.sumocfg", "--time-to-teleport", "-1"])
    
    start = time.time()
    
    # simulation
    while traci.simulation.getMinExpectedNumber() > 0:
        traci.simulationStep()
        if traci.simulation.getTime() == seconds:
            running_time = time.time() - start
            break
        
        for curb in curbs.values():
            
            # colelct neighbor info from last step
            curb._update_neigh_occupy(curbs)
            
            # check if there is any vehicle enters
            v_enter = set(traci.edge.getLastStepVehicleIDs(curb.edge)) - curb.moving_vehicle
            
            # no need to subtract v_leave, because v_leave ideally should be maintained per curb, but now inner loop is on curbs
            # v_enter -= v_leave
            
            for veh in v_enter:
                # new vehicle is for sure not confirmed, no need to check confirmed=True
                # if vehicle has next stops and currect edge is one of them - this vehicle should be considered for current edge
                # also check vehicle is not stopped - during parking right before departure getNextStops() will return original travel plan, which shouldn't happen
                
                if traci.vehicle.getNextStops(veh) and curb.id in [item[2] for item in traci.vehicle.getNextStops(veh)] and not traci.vehicle.isStopped(veh):
                    # if the trips is not ending and the current edge is the parking stop
                    # v_leave should be excluded
                    
                    # vclass: passenger or delivery
                    vclass = traci.vehicle.getVehicleClass(veh)
                    if vclass == 'delivery':
                        occ = curb.dlv_occ
                        cap = curb.dlv_cap
                    else:
                        occ = curb.psg_occ
                        cap = curb.psg_cap
                        
                    if occ < cap:
                        # if can park, add to occupied set : planned + parked
                        curb.occupied_vehicle.add(veh)
                        
                    else:
                        # cannot park at this edge, reroute
                        # item._reroute_choice() returns (curb_id, distance) tuple
                        
                        traci.vehicle.rerouteParkingArea(veh, curb._reroute_choice(veh, curbs)[0])
                        reroute_num += 1
                        
            # v_leave : parked vehicle of last time step - parked vehicle at this time step
            v_leave = curb.parked_vehicle - set(traci.parkingarea.getVehicleIDs(curb.id))
            
            # remove v_leave from occupied set : planned + parked
            curb.occupied_vehicle -= v_leave
            
            # update parked vehicle set
            curb.parked_vehicle = set(traci.parkingarea.getVehicleIDs(curb.id))
            
            # update moving vehicles on the hosting edge
            curb.moving_vehicle = set(traci.edge.getLastStepVehicleIDs(curb.edge))
            
            # update two-type occupancy for next time step
            curb._occupy_cnt()

    # enter False to disconnect before SUMO finishes
    traci.close()

    # flush out intermmediate results
    sys.stdout.flush()
    print(curb.neighbor)
    
    return reroute_num, running_time

In [4]:
def simulate(seconds = 3600, gui = False):
    """
    Main function that controls the simulation
    """

    road_network = utils.create_graph('seattle.net.xml')
    
    # create agent curbs
    curb_ids = []
    root = ET.parse(add_xml).getroot()
    for child in root.iter('additional'):
        for kid in child.iter('parkingArea'):
            curb_ids.append(kid.get('id'))
    
    curbs = []
    for curb_id in curb_ids:
        curbs.append(curbside.SmartCurbside(1, 'seattle.add.xml', 'seattle.net.xml', curb_id, ['passenger', 'taxi'], road_network))
        
    for curb in curbs:
        curb.find_neighborhood(road_network)
    
    reroute_num = 0

    if gui:
        sumoBinary = checkBinary('sumo-gui')
    else:
        sumoBinary = checkBinary('sumo')

    traci.start([sumoBinary, "-c", "seattle.sumocfg"])
    

    control_start_time = 60 # arranging existing vehicle until this time (seconds) to start to control curbs or collecting training samples
    
    control_time_window = 60 # take action every 60 seconds
    
    step = 0 - control_start_time
    MAX_step = control_start_time + control_start_time
    
    # simulation
    start = time.time()
    while traci.simulation.getMinExpectedNumber() > 0 and step < MAX_step:
        traci.simulationStep()
        step += 1
        
        for curb in curbs:
            # update neighbors info
            curb._update_nearby_curb()
            
            # check if there is any vehicle enters
            v_enter = set(traci.edge.getLastStepVehicleIDs(curb.edge)) - curb.moving_vehicle
            
            # no need to subtract v_leave, because v_leave ideally should be maintained per curb, but now inner loop is on curbs
            # v_enter -= v_leave
            
            for veh in v_enter:
                # new vehicle is for sure not confirmed, no need to check confirmed=True
                # if vehicle has next stops and currect edge is one of them - this vehicle should be considered for current edge
                # also check vehicle is not stopped - during parking right before departure getNextStops() will return original travel plan, which shouldn't happen
                if traci.vehicle.getNextStops(veh) and curb.id in [item[2] for item in traci.vehicle.getNextStops(veh)] and not traci.vehicle.isStopped(veh):
                    # if the trips is not ending and the current edge is the parking stop
                    # v_leave should be excluded
                    if any(traci.vehicle.getVehicleClass(veh) in s for s in curb.vclass) and len(curb.occupied_vehicle) < curb.capacity:
                        # if can park, add to occupied set : planned + parked
                        curb.occupied_vehicle.add(veh)
                    else:
                        # cannot park at this edge, reroute
                        # item._reroute_choice() returns (curb_id, distance) tuple
                        traci.vehicle.rerouteParkingArea(veh, curb._reroute_choice()[0])
                        reroute_num += 1
                        
            # v_leave : parked vehicle of last time step - parked vehicle at this time step
            v_leave = curb.parked_vehicle - set(traci.parkingarea.getVehicleIDs(curb.id))
            # remove v_leave from occupied set : planned + parked
            curb.occupied_vehicle -= v_leave
            # update parked vehicle set
            curb.parked_vehicle = set(traci.parkingarea.getVehicleIDs(curb.id))
            # update moving vehicles on the hosting edge
            curb.moving_vehicle = set(traci.edge.getLastStepVehicleIDs(curb.edge))
            
            if step % control_time_window == 0: 
                if step == 0:
                    c_p, o_p, c_d, o_d = curb.cap_passenger, curb.occ_passenger, curb.cap_delivery, curb.occ_passenger
                    state = [c_p, o_p, c_d, o_d]
                    action = curb.policy(state)
                    
                else:
            
            action = 
            
        
        # calculate running time of an interation
        iteration_time = time.time() - start
            
    # enter False to disconnect before SUMO finishes
    traci.close()

    # flush out intermmediate results
    sys.stdout.flush()
    
    return reroute_num, running_time